In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [63]:
df = pd.read_csv("/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv", delimiter=",")
df2 = pd.read_csv("/kaggle/input/spotify-12m-songs/tracks_features.csv")

In [64]:
print(df.shape)
df.head()

(57650, 4)


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [65]:
print(df2.shape)
df2.head()

(1204025, 24)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [70]:
song_to_id_dict = {}
song_names = list(df2["name"])
song_ids = list(df2["id"])

for index in range(len(song_names)):
    song_to_id_dict[song_names[index]] = song_ids[index]

In [85]:
df_new_col = df
df_new_col["song_id"] = df["song"].apply(lambda x: song_to_id_dict[x] if x in song_to_id_dict else np.nan)
df_new_col = df_new_col.dropna(axis=0)

In [5]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()
stop = stopwords.words('english')

def text_processing(text):
    text = [porter.stem(word) for word in text.split()]
    text_filter = [word for word in text if word not in stop]
    return text_filter

df["text"] = df["text"].apply(text_processing)
df_new_col["text"] = df_new_col["text"].apply(text_processing)

In [116]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
common_texts = df["text"].values
artists = df["artist"].values
songs = df["song"].values

documents = [TaggedDocument(doc, [songs[i] + "|" + artists[i]]) for i, doc in enumerate(common_texts)]

common_texts_id = df_new_col["text"].values
artists_id = df_new_col["artist"].values
song_id = df_new_col["song_id"].values
documents_id = [TaggedDocument(doc, [song_id[i]]) for i, doc in enumerate(common_texts_id)]

In [117]:
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model_id = Doc2Vec(documents_id, vector_size=100, window=5, min_count=1, workers=4, epochs=20)

In [118]:
test = """Ribeye steaks are mostly composed of the longissimus dorsi muscle but also contain the complexus and spinalis muscles. The longissimus dorsi is also referred to as the "eye of the ribeye". The spinalis is also referred to as the "ribeye cap" and the complexus is a small muscle at the front of the ribeye which may be trimmed off by the butcher."""

test = text_processing(test)
new_doc_vector = model.infer_vector(test)
similar_documents = model.dv.most_similar(new_doc_vector, topn=5)

new_doc_vector_id = model_id.infer_vector(test)
similar_documents_id = model_id.dv.most_similar(new_doc_vector, topn=5)

In [120]:
similar_documents_id

[('5taUV8kbsjppc3Krz3KgFr', 0.3686203360557556),
 ('4HOihb9lb2cwmUd8UVm3zU', 0.3671843409538269),
 ('5c0d4xAPSMBd7gBjghSqfa', 0.36645933985710144),
 ('6ACepznmEjCUFXyN6y5L8F', 0.3647983968257904),
 ('443Gsrw2ku9g8gwjrYjxnP', 0.36097732186317444)]

In [122]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
filename = 'finalized_model2.sav'
pickle.dump(model2, open(filename, 'wb'))

In [ ]:
%%time
import json
import requests


url = 'https://pbbo1qd4je.execute-api.us-east-2.amazonaws.com/Prod/recommend/'
myobj = {"message": """

The Vietnam War (also known by other names) was a conflict in Vietnam, Laos, and Cambodia from 1 November 1955[A 1] to the fall of Saigon on 30 April 1975.

"""}
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

x = requests.post(url, json = myobj, headers=headers)

print(x.json())